<a href="https://colab.research.google.com/github/howard1028/Advance_machine_learning/blob/main/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hw1-2

import torch
import torchvision
import torch.nn as nn


In [ ]:
# GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU state:', device)  

GPU state: cuda:0


In [ ]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary


# GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU state:', device)


# Cifar-10 data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


# Data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
trainLoader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=0)
testLoader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False, num_workers=0)


# Data classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



GPU state: cuda:0
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Model structure
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = (7,7), stride = (1, 1), padding = (3, 3))
        self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = (5,5), stride = (1, 1), padding = (2, 2))
        
        self.max_pooling = nn.MaxPool2d(kernel_size = (2, 2),stride = (2, 2))
        # self.dropout = nn.Dropout(p = 0.1)
        
        # self.padding1 = nn.ZeroPad2d(8)
        # self.padding2 = nn.ZeroPad2d((9, 8, 1, 1))

        self.fc1 = nn.Linear(128*8*8, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        #self.Softmax = nn.Softmax(dim = 1)


    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.max_pooling(x)
        # x = self.dropout(x)

        x = self.conv2(x)
        # x = self.padding2(x)
        x = F.relu(x)
        x = self.max_pooling(x)
        # x = self.padding1(x)
        # x = self.dropout(x)

        x = x.view(-1, 128*8*8) # -1 -> 不指定flatten的大小

        x = self.fc1(x)
        x = F.relu(x)
        # x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        # x = self.dropout(x)
        x = self.fc3(x)
        #x = self.Softmax(x)

        return x

net = Model().to(device)
print(net) # 印出神經網路的相關資訊

Model(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (max_pooling): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


In [ ]:

# Parameters
criterion = nn.CrossEntropyLoss() # 使用 cross entropy作為loss function之方式
lr = 0.001 # learning rate
epochs = 10 # 訓練3次
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)



# Train
for epoch in range(epochs):
    running_loss = 0.0

    for times, data in enumerate(trainLoader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        if times % 100 == 99 or times+1 == len(trainLoader):
            print('[%d/%d, %d/%d] loss: %.3f' % (epoch+1, epochs, times+1, len(trainLoader), running_loss/100))
            running_loss = 0

print('Finished Training\n')

[1/10, 100/6250] loss: 0.201
[1/10, 200/6250] loss: 0.137
[1/10, 300/6250] loss: 0.196
[1/10, 400/6250] loss: 0.159
[1/10, 500/6250] loss: 0.204
[1/10, 600/6250] loss: 0.153
[1/10, 700/6250] loss: 0.180
[1/10, 800/6250] loss: 0.150
[1/10, 900/6250] loss: 0.181
[1/10, 1000/6250] loss: 0.175
[1/10, 1100/6250] loss: 0.143
[1/10, 1200/6250] loss: 0.153
[1/10, 1300/6250] loss: 0.130
[1/10, 1400/6250] loss: 0.179
[1/10, 1500/6250] loss: 0.151
[1/10, 1600/6250] loss: 0.162
[1/10, 1700/6250] loss: 0.139
[1/10, 1800/6250] loss: 0.148
[1/10, 1900/6250] loss: 0.174
[1/10, 2000/6250] loss: 0.154
[1/10, 2100/6250] loss: 0.174
[1/10, 2200/6250] loss: 0.169
[1/10, 2300/6250] loss: 0.216
[1/10, 2400/6250] loss: 0.201
[1/10, 2500/6250] loss: 0.218
[1/10, 2600/6250] loss: 0.156
[1/10, 2700/6250] loss: 0.178
[1/10, 2800/6250] loss: 0.171
[1/10, 2900/6250] loss: 0.195
[1/10, 3000/6250] loss: 0.197
[1/10, 3100/6250] loss: 0.193
[1/10, 3200/6250] loss: 0.195
[1/10, 3300/6250] loss: 0.200
[1/10, 3400/6250] l

In [ ]:

# Test
# top 1 和 top 5的結果
top1_correct = 0
top5_correct = 0
total = 0
with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = net(inputs)
        labels_resize = labels.view(-1, 1)

        _, top1_predicted = torch.max(outputs.data, 1)
        _, top5_predicted = outputs.topk(5, 1, True, True)

        total += labels.size(0)
        top1_correct += (top1_predicted == labels).sum().item()
        top5_correct += (top5_predicted == labels_resize).sum().item()

print('[Top 1] Accuracy of the network on the 10000 test inputs: %d %%' % (100 * top1_correct / total))
print('[Top 5] Accuracy of the network on the 10000 test inputs: %d %%' % (100 * top5_correct / total))

# 每個 class 的準確度
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(8):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

summary((net), (3, 32, 32))

[Top 1] Accuracy of the network on the 10000 test inputs: 72 %
[Top 5] Accuracy of the network on the 10000 test inputs: 97 %
Accuracy of plane : 72 %
Accuracy of   car : 85 %
Accuracy of  bird : 64 %
Accuracy of   cat : 54 %
Accuracy of  deer : 73 %
Accuracy of   dog : 58 %
Accuracy of  frog : 81 %
Accuracy of horse : 77 %
Accuracy of  ship : 83 %
Accuracy of truck : 72 %
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           9,472
         MaxPool2d-2           [-1, 64, 16, 16]               0
            Conv2d-3          [-1, 128, 16, 16]         204,928
         MaxPool2d-4            [-1, 128, 8, 8]               0
            Linear-5                  [-1, 128]       1,048,704
            Linear-6                   [-1, 64]           8,256
            Linear-7                   [-1, 10]             650
Total params: 1,272,010
Trainable params: 1,272